In [1]:
import numpy as np
import tensorflow as tf

from recommender import Recommender
import tensorflow_recommenders as tfrs
import pandas as pd

from item_embedding import ItemModel


In [2]:
test_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test").cache()

portfolios = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/portfolios_tfds").cache()

items_ids = portfolios.batch(10000).map(lambda x: x["STOCKCODE"])
item_names = portfolios.batch(10000).map(lambda x: x["STOCKNAME"])
item_GICS = portfolios.batch(10000).map(lambda x: x["GICS"])

user_ids = portfolios.batch(10000).map(lambda x: x["CDSACCNO"])

unique_item_ids = np.unique(np.concatenate(list(items_ids)))
unique_item_names = np.unique(np.concatenate(list(item_names)))
unique_item_gics = np.unique(np.concatenate(list(item_GICS)))

unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# need these to initialize timestamp embedding layers in future steps

timestamps = np.concatenate(list(portfolios.map(lambda x: x["UNIX_TS"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:
len(portfolios)

3847

In [42]:
from recommender import Recommender

loaded_model = Recommender(
    use_timestamp = True,
    portfolios = portfolios
)


In [45]:
loaded_model.load_weights("D:/dev work/recommender systems/ATRAD_CARS/model_weights/2024_04_29/tf_retrival_2024_04_29_13_31")

loaded_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

loaded_model.evaluate(test_ds.shuffle(100_000).batch(1024), return_dict=True)

1/1 [==============================] - 0s 411ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0182 - factorized_top_k/top_5_categorical_accuracy: 0.0195 - factorized_top_k/top_10_categorical_accuracy: 0.0195 - factorized_top_k/top_50_categorical_accuracy: 0.0416 - factorized_top_k/top_100_categorical_accuracy: 0.0819 - loss: 5169.8188 - regularization_loss: 0.0000e+00 - total_loss: 5169.8188


{'factorized_top_k/top_1_categorical_accuracy': 0.018205462023615837,
 'factorized_top_k/top_5_categorical_accuracy': 0.019505850970745087,
 'factorized_top_k/top_10_categorical_accuracy': 0.019505850970745087,
 'factorized_top_k/top_50_categorical_accuracy': 0.04161248356103897,
 'factorized_top_k/top_100_categorical_accuracy': 0.08192458003759384,
 'loss': 5169.81884765625,
 'regularization_loss': 0,
 'total_loss': 5169.81884765625}

In [46]:
1.58641972e-02

0.0158641972

In [47]:
trainable_variables = loaded_model.trainable_variables  # Replace 'retriever' with your model instance

# # Count the total number of parameters
# total_params = sum(param.num_elements for param in trainable_variables)

# print(f"Total Trainable Parameters: {total_params}")

trainable_variables[1]

<tf.Variable 'embedding_9/embeddings:0' shape=(34, 8) dtype=float32, numpy=
array([[ 1.58641972e-02, -3.76168862e-02,  2.27185749e-02,
         3.82469781e-02,  2.58139856e-02, -8.58383253e-03,
        -3.82197984e-02, -3.44697237e-02],
       [-4.76456583e-02,  1.89965412e-01, -5.21973252e-01,
         4.70650941e-02,  7.25552738e-02,  2.62920223e-02,
        -2.83564180e-01, -4.05938961e-02],
       [-3.52241695e-01, -5.55856228e-01,  2.87445802e-02,
         1.65968776e-01, -1.03208810e-01,  2.90869772e-01,
        -4.84446794e-01, -2.33215779e-01],
       [-1.25361979e+00,  1.01868367e+00,  2.22905323e-01,
        -7.14606941e-01,  5.08995295e-01,  3.02225351e-01,
         5.96825600e-01,  3.49049211e-01],
       [ 3.29938382e-01,  1.67680293e-01,  2.82983214e-01,
        -2.96394765e-01, -6.10856041e-02, -3.13666947e-02,
         3.21693689e-01,  2.91164994e-01],
       [-5.70660353e-01, -1.09546983e+00,  4.64174896e-01,
        -7.35128284e-01,  7.74168849e-01,  7.96196699e-01,
 

In [6]:
items = portfolios.map(lambda x: {
'STOCKCODE' : x['STOCKCODE'],
'STOCKNAME' : x['STOCKNAME'],
'GICS' : x['GICS'],
})

In [7]:
stock_info = pd.read_excel('../../data/stock_data.xlsx')
stock_info = stock_info.drop(['Unnamed: 0','buisnesssummary'],axis = 1)
stock_info.shape

(282, 3)

In [8]:
stock_info.tail()

,symbol,name,gics_code
277,HPWR,RESUS ENERGY PLC,Utilities
278,VLL,VIDULLANKA PLC,Utilities
279,VONE,VALLIBEL ONE PLC,Utilities
280,LGIL,LOLC GENERAL INSURANCE PLC,NaN
281,NAVF,NAMAL ACUITY VALUE FUND,NaN


In [9]:
stock_info = stock_info.rename(columns = {
    'symbol':'STOCKCODE',
    'name' : 'STOCKNAME',
    'gics_code' : 'GICS'
})

In [10]:
list(unique_item_gics)

[b'45103010 - Application Software',
 b'Automobiles & Components',
 b'Banks',
 b'Banks Finance & Insurance',
 b'Capital Goods',
 b'Commercial & Professional Services',
 b'Consumer Durables & Apparel',
 b'Consumer Finance',
 b'Consumer Services',
 b'Diversified Financial Services',
 b'Diversified Financials',
 b'Energy',
 b'FOOD & STAPLES RETAILING',
 b'FOOD BEVERAGE & TOBACCO',
 b'Food Beverage & Tobacco',
 b'Health Care Equipment & Services',
 b'Household & Personal Products',
 b'Industrials',
 b'Insurance',
 b'Investment Banking & Brokerage',
 b'Materials',
 b'Materials (1510)',
 b'Multi-line Insurance',
 b'Power and Energy',
 b'Real Estate',
 b'Real Estate (6010)',
 b'Real Estate Development',
 b'Retailing',
 b'Services',
 b'Telecommunication Services',
 b'Trading',
 b'Transpotation',
 b'Utilities']

In [11]:
stock_info = stock_info[~stock_info['GICS'].isna()]

In [12]:
stock_info.head(2)

,STOCKCODE,STOCKNAME,GICS
0,HBS,hSenid Business Solutions PLC,45103010 - Application Software
1,TYRE,KELANI TYRES PLC,Automobiles & Components


In [13]:
stock_info = stock_info.drop_duplicates()

items_ds = tf.data.Dataset.from_tensor_slices(stock_info.to_dict(orient= 'list'))

In [14]:
next(iter(items_ds))

{'STOCKCODE': <tf.Tensor: shape=(), dtype=string, numpy=b'HBS'>,
 'STOCKNAME': <tf.Tensor: shape=(), dtype=string, numpy=b'hSenid Business Solutions PLC'>,
 'GICS': <tf.Tensor: shape=(), dtype=string, numpy=b'45103010 - Application Software'>}

In [15]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(loaded_model.user_model)

In [16]:
test_item_model_mapper = ItemModel(
    unique_item_ids,
    unique_item_names,
    unique_item_gics,
    # map_ = True
)

In [17]:
mapped_items = items_ds.batch(len(items_ds)).map(lambda x : test_item_model_mapper(x, map_ = True))

In [18]:
mapped_items_tensor = next(iter(mapped_items)) #.numpy()

In [19]:
mapped_items_tensor.shape

TensorShape([280, 32])

In [20]:
items_identiifiers = items_ds.map(lambda x: x["STOCKCODE"])
items_identiifiers = next(iter(items_identiifiers.batch(len(items_identiifiers))))
items_identiifiers.shape

TensorShape([280])

In [21]:
# recommends item out of the entire items dataset.
# index.index(items.batch(100).map(loaded_model.item_model), items)

index.index(mapped_items_tensor, items_identiifiers)

In [22]:
unique_user_ids[:2]


array([b'HNB-254-LC/00', b'RPS-13617-LI/00'], dtype=object)

In [23]:
str('HNB-254-LC/00')

'HNB-254-LC/00'

In [24]:
1.64199919e+09, 1.64207011e+09, 1.64214102e+09

(1641999190.0, 1642070110.0, 1642141020.0)

In [55]:
# # recommends item out of the entire items dataset.
# index.index(items.batch(100).map(lambda x : test_item_model_mapper(x, map = True)), items)

# Get recommendations.
user_id = str('HNB-254-LC/00')
_, titles = index(
    (
        tf.constant([user_id]),
        tf.constant([1.64199919e+09])
        )
    )
print(f"Recommendations for user %s: {titles[0]}" %(user_id))

Recommendations for user HNB-254-LC/00: [b'COF' b'VONE' b'LLUB' b'LVEF' b'HPWR' b'CPRT' b'AGAL' b'SDB' b'MSL'
 b'TESS']
